In [1]:
from pairwise_model import *
from text_utils import *
import regex as re
from bm25_utils import BM25Gensim
from qa_model import *
from tqdm.auto import tqdm
tqdm.pandas()

Note: to be able to use all crisp methods, you need to install some additional packages:  {'wurlitzer', 'leidenalg', 'graph_tool', 'infomap', 'karateclub'}
Note: to be able to use all overlapping methods, you need to install some additional packages:  {'ASLPAw', 'karateclub'}
Note: to be able to use all bipartite methods, you need to install some additional packages:  {'wurlitzer', 'leidenalg', 'infomap'}


In [2]:
df_wiki_windows = pd.read_csv("./data/wikipedia_20220620_cleaned_v2.csv")
df_wiki = pd.read_csv("./data/wikipedia_20220620_short.csv")
df_wiki.title = df_wiki.title.apply(str)

In [3]:
entity_dict = json.load(open("./data/entities.json"))
new_dict = dict()
for key, val in entity_dict.items():
    val = val.replace("wiki/", "").replace("_", " ")
    entity_dict[key] = val
    key = preprocess(key)
    new_dict[key.lower()] = val
entity_dict.update(new_dict)
title2idx = dict([(x.strip(), y) for x, y in zip(df_wiki.title, df_wiki.index.values)])

In [4]:
qa_model = QAEnsembleModel("nguyenvulebinh/vi-mrc-large", ["./data/qa_model_robust.bin"], entity_dict)
pairwise_model_stage1 = PairwiseModel("nguyenvulebinh/vi-mrc-base").half()
pairwise_model_stage1.load_state_dict(torch.load("./data/pairwise_v2.bin"))
pairwise_model_stage1.eval()

pairwise_model_stage2 = PairwiseModel("nguyenvulebinh/vi-mrc-base").half()
pairwise_model_stage2.load_state_dict(torch.load("./data/pairwise_stage2_seed0.bin"))

Some weights of the model checkpoint at nguyenvulebinh/vi-mrc-base were not used when initializing RobertaModel: ['qa_outputs.bias', 'qa_outputs.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at nguyenvulebinh/vi-mrc-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at nguyenvulebinh/vi-mrc-base were not used w

<All keys matched successfully>

In [5]:
bm25_model_stage1 = BM25Gensim("./data/bm25_stage1/", entity_dict, title2idx)
bm25_model_stage2_full = BM25Gensim("./data/bm25_stage2/full_text/", entity_dict, title2idx)
bm25_model_stage2_title = BM25Gensim("./data/bm25_stage2/title/", entity_dict, title2idx)

In [28]:
def get_answer_e2e(question):
    #Bm25 retrieval for top200 candidates
    query = preprocess(question).lower()
    top_n, bm25_scores = bm25_model_stage1.get_topk_stage1(query, topk=200)
    titles = [preprocess(df_wiki_windows.title.values[i]) for i in top_n]
    texts = [preprocess(df_wiki_windows.text.values[i]) for i in top_n]
    
    #Reranking with pairwise model for top10
    question = preprocess(question)
    ranking_preds = pairwise_model_stage1.stage1_ranking(question, texts)
    ranking_scores = ranking_preds * bm25_scores
    
    #Question answering
    best_idxs = np.argsort(ranking_scores)[-10:]
    ranking_scores = np.array(ranking_scores)[best_idxs]
    texts = np.array(texts)[best_idxs]
    best_answer = qa_model(question, texts, ranking_scores)
    if best_answer is None:
        return "Chịu"
    bm25_answer = preprocess(str(best_answer).lower(), max_length=128, remove_puncts=True)
    
    #Entity mapping
    if not check_number(bm25_answer):
        bm25_question = preprocess(str(question).lower(), max_length=128, remove_puncts=True)
        bm25_question_answer = bm25_question + " " + bm25_answer
        candidates, scores = bm25_model_stage2_title.get_topk_stage2(bm25_answer, raw_answer=best_answer)
        titles = [df_wiki.title.values[i] for i in candidates]
        texts = [df_wiki.text.values[i] for i in candidates]
        ranking_preds = pairwise_model_stage2.stage2_ranking(question, best_answer, titles, texts)
        if ranking_preds.max() >= 0.1:
            final_answer = titles[ranking_preds.argmax()]
        else:
            candidates, scores = bm25_model_stage2_full.get_topk_stage2(bm25_question_answer)
            titles = [df_wiki.title.values[i] for i in candidates] + titles
            texts = [df_wiki.text.values[i] for i in candidates] + texts
            ranking_preds = np.concatenate(
                [pairwise_model_stage2.stage2_ranking(question, best_answer, titles, texts), ranking_preds])
        final_answer = "wiki/"+titles[ranking_preds.argmax()].replace(" ","_")
    else:
        final_answer = bm25_answer.lower()
    return final_answer

In [7]:
get_answer_e2e("Ai đẹp nhất Zalo")

'Chịu'

In [11]:
get_answer_e2e("Nhudq đang làm việc ở đâu")

'wiki/Thanh_Hóa'

In [12]:
get_answer_e2e("Ai là tác giả Lord of the Rings")

'wiki/J._R._R._Tolkien'

In [34]:
qa_model.thr = 0.05

In [62]:
get_answer_e2e("thủ tướng Nhật Bản là ai?")

'wiki/Kiyoura_Keigo'